# Single image processing

In [1]:
import ee
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm, gamma, f, chi2
import IPython.display as disp
%matplotlib inline

# Trigger the authentication flow.
ee.Authenticate()
 
# Initialize the library.
ee.Initialize()

Enter verification code:  4/1AfgeXvvkaf-12N-30gXZCEwXb_xvxON6VdUj4B8iY-AjkPBcKeNQMr-48-s



Successfully saved authorization token.


In [16]:
# Define area of interest
# If you have a GeoJSON file, copy paste.
# If you have a KML, export to GeoJSON (plenty of free tools online)
# or retrieve P

geoJSON = {
    "type": "FeatureCollection",
    "name": "Budrio_half-right",
    "crs": { "type": "name", "properties": { "name": "urn:ogc:def:crs:OGC:1.3:CRS84" } },
    "features": [
        { "type": "Feature", "properties": { "Name": "Budrio_campo_right", "description": None, "tessellate": 1 }, "geometry": { "type": "Polygon", "coordinates": [ [ [ 11.533083525108189, 44.570697786985733 ], [ 11.532456431504921, 44.569896556005418 ], [ 11.53276321111148, 44.569791821233608 ], [ 11.53338591526418, 44.570601794304793 ], [ 11.533083525108189, 44.570697786985733 ] ] ] } },
        { "type": "Feature", "properties": { "Name": "Budrio_campo_safe_half", "description": None, "tessellate": 1 }, "geometry": { "type": "Polygon", "coordinates": [ [ [ 11.53262979564736, 44.570842547510622 ], [ 11.532328100248961, 44.570445732016537 ], [ 11.53264162483709, 44.570339694294631 ], [ 11.532950828277439, 44.570738040751841 ], [ 11.53262979564736, 44.570842547510622 ] ] ] } }
    ]
}
coords = [geoJSON['features'][i]['geometry']['coordinates'] for i in range(2)]
aoi = ee.Geometry.MultiPolygon(coords)

# Filters definition
fil_orb95   = ee.Filter.eq('relativeOrbitNumber_start', 95)
fil_orb168  = ee.Filter.eq('relativeOrbitNumber_start', 168)
fil_orb117 = ee.Filter.eq('relativeOrbitNumber_start', 117)

sp17 = ee.Filter.date('2017-04-04', '2017-05-22')
su17 = ee.Filter.date('2017-05-22', '2017-09-15')
au17 = ee.Filter.date('2017-09-15', '2017-11-02')
tot = ee.Filter.date('2014-10-03', '2022-12-01')
tot1 = ee.Filter.date('2014-10-03', '2016-09-30')
tot2 = ee.Filter.date('2016-10-02', '2022-12-01')

# Get collection of images and filter
img = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT') # linear scale for mean, var computation
        .filterBounds(aoi)
        .filter(tot2)
        .sort('system:time_start'))
        # .select('VV')

# acq_times = img.aggregate_array('system:time_start').getInfo()
# len([time.strftime('%x', time.gmtime(acq_time/1000)) for acq_time in acq_times])

# To get all bands and infos of img collection, use
# img.getInfo()

In [10]:
# Extract data
geometry_title = input('Please provide a title for AoI geometry. (Default: Budrio_half-right)')
if not geometry_title: geometry_title='Budrio_half-right'

print('Mean is computed by spatial average in linear scale.\n'+
      'Std is the square root of variance in linear scale, '+
      'transformed in dB by mantaining constant relative error.')

#-----------------------------------------------------------------------------

def lin_db(x):
    return 10*np.log10(x)

def db_lin(x):
    return 10**(x/10)

#-----------------------------------------------------------------------------

def extract_data(image:ee.Image):
    """Ausiliary function to extract data from an Image
    
    This function extracts spatial means and std.dev
    via spatial reducers (reduceRegion).
    Optimal implementation is to map this function
    on a whole ImageCollection via .map() and insert the
    return into a ee.FeatureCollection.
    
    Return
    ------
    ee.Feature
    
    """
    
    mean = image.reduceRegion(**{ 
        'reducer': ee.Reducer.mean(),
        'geometry': aoi,
    })
    
    dev = image.reduceRegion(**{ 
        'reducer': ee.Reducer.stdDev(),
        'geometry': aoi,
    })
    
    var = image.reduceRegion(**{
        'reducer':ee.Reducer.variance(),
        'geometry': aoi,
    })
        
    properties = {
        'Date': image.get('system:time_start'), # only way to get a timestr is an external operation
        'Geometry': geometry_title,
        'VV[lin]': mean.get('VV'),
        'VH[lin]': mean.get('VH'),
        'Angle[°]': mean.get('angle'),
        'VV_var[lin]': var.get('VV'),
        'VH_var[lin]': var.get('VH'),
        'Orb': image.get('relativeOrbitNumber_start'),
        'Pass': image.get('orbitProperties_pass'),
    }
    return ee.Feature(None, properties)

#-----------------------------------------------------------------------------

Please provide a title for AoI geometry. (Default: Budrio_half-right) 


Mean is computed by spatial average in linear scale.
Std is the square root of variance in linear scale, transformed in dB by mantaining constant relative error.


In [17]:
data = ee.FeatureCollection(img.map(extract_data))

data_out = data.getInfo()
# data_out.keys()
# data_out.get('features')[0].get('properties')
data_out_to_df = [e.get('properties') for e in data_out.get('features')]; data_out_to_df[0]

{'Angle[°]': 37.385643005371094,
 'Date': 1475427979470,
 'Geometry': 'Budrio_half-right',
 'Orb': 117,
 'Pass': 'ASCENDING',
 'VH[lin]': 0.029780330376576295,
 'VH_var[lin]': 0.00013057202500957527,
 'VV[lin]': 0.11276936047056263,
 'VV_var[lin]': 0.001706829189185973}

In [18]:
df = pd.DataFrame.from_dict(data_out_to_df)

def clean_date(date:int):
    return time.strftime('%x %H', time.localtime((date)/1000))

df.Date = df.Date.apply(lambda x : pd.to_datetime(clean_date(x)))

df['VV[dB]'] = df['VV[lin]'].apply(lambda x : lin_db(x))
df['VH[dB]'] = df['VH[lin]'].apply(lambda x : lin_db(x))
df['VV_var[dB]'] = df['VV_var[lin]']/df['VV[lin]']*(10/np.log(10))
df['VH_var[dB]'] = df['VH_var[lin]']/df['VH[lin]']*(10/np.log(10))

df

,Angle[°],Date,Geometry,Orb,Pass,VH[lin],VH_var[lin],VV[lin],VV_var[lin],VV[dB],VH[dB],VV_var[dB],VH_var[dB]
0,37.385643,2016-10-02 19:00:00,Budrio_half-right,117,ASCENDING,0.029780,0.000131,0.112769,0.001707,-9.478089,-15.260705,0.065733,0.019042
1,31.274052,2016-10-06 07:00:00,Budrio_half-right,168,DESCENDING,0.037527,0.000418,0.306743,0.008070,-5.132247,-14.256618,0.114251,0.048350
2,41.286919,2016-10-07 07:00:00,Budrio_half-right,95,DESCENDING,0.014642,0.000043,0.182180,0.006520,-7.395002,-18.343923,0.155435,0.012692
3,37.360371,2016-10-08 19:00:00,Budrio_half-right,117,ASCENDING,0.009171,0.000009,0.093657,0.000342,-10.284586,-20.375987,0.015882,0.004293
4,31.349546,2016-10-12 07:00:00,Budrio_half-right,168,DESCENDING,0.023354,0.000072,0.241720,0.005393,-6.166868,-16.316429,0.096904,0.013302
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1009,37.399632,2022-11-12 18:00:00,Budrio_half-right,117,ASCENDING,0.008295,0.000025,0.062336,0.000590,-12.052590,-20.812039,0.041081,0.013176
1010,31.267229,2022-11-16 06:00:00,Budrio_half-right,168,DESCENDING,0.024676,0.000096,0.215107,0.006513,-6.673451,-16.077300,0.131493,0.016926
1011,41.307030,2022-11-23 06:00:00,Budrio_half-right,95,DESCENDING,0.026612,0.000089,0.244474,0.006738,-6.117674,-15.749171,0.119698,0.014454
1012,37.400272,2022-11-24 18:00:00,Budrio_half-right,117,ASCENDING,0.028167,0.000099,0.181575,0.002180,-7.409432,-15.502580,0.052148,0.015202


In [25]:
dftot = pd.concat([df1,df2]).set_index('Date')
dftot.to_csv(f'..\Data\Golden_GEE_2014-22.csv', sep = '\t')

# Merging with in-situ data

Ref: Building_Golden.ipynb

In [31]:
# Platinum_Budrio has 2 sheets, one for 2017, one for 2020

import datetime as dtt

database = pd.ExcelFile('..\\Data\\Platinum_Budrio.xlsx', engine='openpyxl')

data_1h_2017 = database.parse('2017_1h')
data_1h_2017['Ora'] = pd.to_datetime(data_1h_2017['Ora'].astype('str')).apply(lambda x: x.time())
data_1h_2017['BiOra'] = data_1h_2017['Ora'].apply(lambda x: 2*np.around(x.hour/2))
data_1h_2017['Data'] = pd.to_datetime(data_1h_2017['Data'].astype('str')).apply(lambda x: x.date())
data_1h_2017['Mese'] = data_1h_2017['Data'].apply(lambda x: x.month)
data_1h_2017['Date'] = data_1h_2017.apply(lambda r : dtt.datetime.combine(r['Data'],r['Ora']),1)
data_1h_2017 = data_1h_2017.drop(['ID', 'Data', 'Ora', '214Pb[cps]', 'BiOra', 'Mese'],axis=1)
data_1h_2017

,SWC[m3/m3],Pioggia[mm],Irrigazione[mm],Temperatura[°C],Date
0,0.173254,0.0,0.0,17.4097,2017-04-03 11:00:00
1,0.174514,0.0,0.0,19.1982,2017-04-03 12:00:00
2,0.178998,0.0,0.0,20.9032,2017-04-03 13:00:00
3,0.206764,0.0,0.0,21.8341,2017-04-03 14:00:00
4,0.169945,0.0,0.0,22.3737,2017-04-03 15:00:00
...,...,...,...,...,...
5120,0.139442,0.0,0.0,11.3791,2017-11-02 19:00:00
5121,0.135318,0.0,0.0,10.9391,2017-11-02 20:00:00
5122,0.149532,0.0,0.0,10.7549,2017-11-02 21:00:00
5123,0.130251,0.0,0.0,11.1445,2017-11-02 22:00:00


In [32]:
golden = pd.merge(left=df, right=data_1h_2017, how='left', on='Date').iloc[:-3]; golden

,Angle[°],Date,Geometry,Orb,Pass,VH[lin],VH_var[lin],VV[lin],VV_var[lin],VV[dB],VH[dB],VV_var[dB],VH_var[dB],SWC[m3/m3],Pioggia[mm],Irrigazione[mm],Temperatura[°C]
0,31.282551,2017-04-04 07:00:00,Budrio_half-right,168,DESCENDING,0.009136,0.000018,0.070329,0.000630,-11.528672,-20.392462,0.038881,0.008441,0.170173,0.0,0.0,8.7339
1,41.103306,2017-04-05 07:00:00,Budrio_half-right,95,DESCENDING,0.006342,0.000010,0.090708,0.001534,-10.423533,-21.977589,0.073466,0.006573,0.147105,0.0,0.0,11.8548
2,37.537476,2017-04-06 19:00:00,Budrio_half-right,117,ASCENDING,0.005942,0.000007,0.062548,0.000390,-12.037891,-22.260785,0.027052,0.004964,0.150527,0.0,0.0,15.6591
3,31.368000,2017-04-10 07:00:00,Budrio_half-right,168,DESCENDING,0.006690,0.000009,0.066282,0.000831,-11.786029,-21.745415,0.054418,0.005718,0.159550,0.0,0.0,8.5507
4,41.305325,2017-04-11 07:00:00,Budrio_half-right,95,DESCENDING,0.005057,0.000007,0.056139,0.000400,-12.507326,-22.961495,0.030916,0.005962,0.160673,0.0,0.0,10.5575
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,41.293633,2017-10-26 07:00:00,Budrio_half-right,95,DESCENDING,0.011202,0.000027,0.093611,0.000985,-10.286710,-19.507143,0.045695,0.010312,0.155637,0.0,0.0,5.1388
103,37.531326,2017-10-27 19:00:00,Budrio_half-right,117,ASCENDING,0.012791,0.000031,0.069743,0.000667,-11.565022,-18.931034,0.041511,0.010434,0.155525,0.0,0.0,14.4822
104,31.386597,2017-10-31 06:00:00,Budrio_half-right,168,DESCENDING,0.010243,0.000017,0.159341,0.004745,-7.976737,-19.895653,0.129336,0.007121,0.167105,0.0,0.0,4.9978
105,41.307350,2017-11-01 06:00:00,Budrio_half-right,95,DESCENDING,0.011606,0.000018,0.090067,0.000921,-10.454340,-19.353215,0.044419,0.006885,0.124052,NaN,0.0,NaN


In [33]:
save = input("Wanna save in Data directory? [y/n] ")
if save=='y': golden.to_csv(f'..\Data\Golden_GEE.csv', sep = '\t')

Wanna save in root directory? [y/n]  y
